# core

> Universal helper functions

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
import pandas as pd

## column helpers

In [ ]:
#| export
def move_columns(
    df: pd.DataFrame,  # Input
    cols_to_move: str,  # Single 
    pos: int  # Target
) -> pd.DataFrame:
    """
    Move one or more columns to a specified position in a DataFrame.
    """

    if isinstance(cols_to_move, str):
        cols_to_move = [cols_to_move]

    cols = list(df.columns)

    for c in cols_to_move:
        cols.remove(c)

    new_cols = cols[:pos] + cols_to_move + cols[pos:]
    return df[new_cols]


In [ ]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
df

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [ ]:
move_columns(df, 'C', 0)

,C,A,B
0,7,1,4
1,8,2,5
2,9,3,6


In [ ]:
sample_string = 'KnotenNr. der Stücklistenposition'

In [ ]:
sample_string = sample_string.lower()
sample_string

'knotennr. der stücklistenposition'

In [ ]:
[c for c in sample_string][:10]

['k', 'n', 'o', 't', 'e', 'n', 'n', 'r', '.', ' ']

In [ ]:
[c.isalnum() for c in sample_string][:10]

[True, True, True, True, True, True, True, True, False, False]

In [ ]:
[c if c.isalnum() else '_' for c in sample_string][:10]

['k', 'n', 'o', 't', 'e', 'n', 'n', 'r', '_', '_']

In [ ]:
sample_string = "".join([c if c.isalnum() else '_' for c in sample_string])
sample_string

'knotennr__der_stücklistenposition'

In [ ]:
sample_string.split('_')

['knotennr', '', 'der', 'stücklistenposition']

In [ ]:
[o for o in filter(None, sample_string.split('_'))]

['knotennr', 'der', 'stücklistenposition']

In [ ]:
'_'.join(filter(None,sample_string.split('_')))

'knotennr_der_stücklistenposition'

In [ ]:
#| export
def clean_string(input_string:str):
    """Cleans input_string"""
    processed_string = "".join(c if c.isalnum() else "_" for c in input_string.lower()).strip("_")
    return "_".join(filter(None, processed_string.split("_")))

In [ ]:
clean_string(sample_string)

'knotennr_der_stücklistenposition'

In [ ]:
#| hide
test_eq(clean_string('___'), '')
test_eq(clean_string('Mat# Desc.'), 'mat_desc')
test_eq(clean_string(''), '')
test_eq(clean_string('987654321'), '987654321')

In [ ]:
#| export
def clean_col_names(df:pd.DataFrame) -> pd.DataFrame:
    """Returns df with clean column names by using `clean_string` on each column name."""
    df.columns = [clean_string(col.lower()) for col in df.columns]
    return df

In [ ]:
#| hide
data = {
    "Cust_ID.": [101, 102, 103, 104, 105],
    "Order--Date": ["2024-02-01", "2024-02-02", "2024-02-03", "2024-02-04", "2024-02-05"],
    "Prdct.Name!": ["Widget A", "Widget B", "Gadget X", "Gadget Y", "Device Z"],
    "QTY___Ordered": [10, 20, 5, 7, 15],
    "Unit$Price": [99.99, 149.99, 249.99, 349.99, 199.99]
}

df = pd.DataFrame(data)

In [ ]:
df.head(2)

,Cust_ID.,Order--Date,Prdct.Name!,QTY___Ordered,Unit$Price
0,101,2024-02-01,Widget A,10,99.99
1,102,2024-02-02,Widget B,20,149.99


In [ ]:
df = clean_col_names(df)
df.head(2)

,cust_id,order_date,prdct_name,qty_ordered,unit_price
0,101,2024-02-01,Widget A,10,99.99
1,102,2024-02-02,Widget B,20,149.99


## validate functions

Tests to check if we use out of scope variables inside function

In [ ]:
a = 10

In [ ]:
def my_sum(b):
    return a + b

my_sum(32)

42

In [ ]:
import inspect

In [ ]:
# inspect.getclosurevars(eval("my_sum"))
inspect.getclosurevars(my_sum)

ClosureVars(nonlocals={}, globals={'a': 10}, builtins={}, unbound=set())

In [ ]:
def my_callback(result):
    print("Cell just finished running!")

get_ipython().events.register('post_run_cell', my_callback)

Cell just finished running!


In [ ]:
get_ipython().events.unregister('post_run_cell', my_callback)

In [ ]:
import ast

Functions defined: ['check_funcs']


In [ ]:
def check_funcs(result):
    tree = ast.parse(result.info.raw_cell)
    func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    if func_names:
        print(f"Functions defined: {func_names}")

In [ ]:
get_ipython().events.register('post_run_cell', check_funcs)

In [ ]:
get_ipython().events.unregister('post_run_cell', check_funcs)

In [ ]:
def check_funcs(result):
    tree = ast.parse(result.info.raw_cell)
    func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    if func_names:
        print(f"Functions defined: {func_names}")

Functions defined: ['check_funcs']


In [ ]:
get_ipython().events.register('post_run_cell', check_funcs)

In [ ]:
get_ipython().events.unregister('post_run_cell', check_funcs)

In [ ]:
import ast
import inspect

In [ ]:
def check_global_deps(result):
    tree = ast.parse(result.info.raw_cell)
    func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    
    ns = get_ipython().user_ns
    for name in func_names:
        if name in ns:
            func = ns[name]
            cv = inspect.getclosurevars(func)
            if cv.globals:
                print(f"⚠️  '{name}' depends on global variables: {list(cv.globals.keys())}")

In [ ]:
get_ipython().events.register('post_run_cell', check_global_deps)

In [ ]:
def my_sum(b):
    return a + b

my_sum(32)

42

⚠️  'my_sum' depends on global variables: ['a']


In [ ]:
get_ipython().events.unregister('post_run_cell', check_global_deps)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()